# KNN benchmark

This notebook repeats the KNN benchmark reported in Smith et al.

In [75]:
import os
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import roc_auc_score, accuracy_score
import torch
from utilities import hdf_keys

In [2]:
DATA_PATH = Path("/data/pfizer_tx")

Benchmark classification tasks only

In [23]:
classification_strings = ['stage', 'grade', 'GSE']
all_classification_files = [f for f in os.listdir(DATA_PATH/"tasks_all_clr")\
                            if any([e in f for e in classification_strings])]

Work up a cross-validated classifier on a trial dataset: `all_clr_train_LUAD_stage.h5`

In [24]:
all_classification_files[0]

'all_clr_train_LUAD_stage.h5'

In [25]:
test_data_path = DATA_PATH/"tasks_all_clr"/all_classification_files[0]
keys = hdf_keys(test_DATA_PATH)
print(keys)
test_data = {key : pd.read_hdf(test_data_path, key = key) for key in keys}

['/expression', '/labels']


In [79]:
outer = KFold(n_splits=5)

In [93]:
def inner_func(input_x, input_y):
    inner = KFold(n_splits=5)
    k = [1, 3, 5, 7, 9] # matches the search used in the original paper
    best_k = None
    best_auc = 0.5
    for idx, (t, v) in enumerate(inner.split(input_x)):
        this_knn = KNN(n_neighbors=k[idx], n_jobs=-1)
        x_train, y_train, x_valid, y_valid = input_x.iloc[t], input_y.iloc[t],\
            input_x.iloc[v], input_y.iloc[v]
        print(f"Fitting model with k: {k[idx]}")
        fitted_model = this_knn.fit(x_train, y_train)
        valid_prob = fitted_model.predict_proba(x_valid)
        valid_auc = roc_auc_score(y_true=y_valid, y_score=valid_prob[:,1])
        if valid_auc > best_auc:
            best_auc = valid_auc
            best_k = k[idx]
        # finally train on all the data and predict held out sample
    print(f"Best K: {best_k}, training final model")
    this_knn = KNN(n_neighbors=best_k, n_jobs=-1)
    fitted_model = this_knn.fit(input_x, input_y)
    return fitted_model

In [94]:
model = inner_func(test_data["/expression"], test_data["/labels"])

Fitting model with k: 1
Fitting model with k: 3
Fitting model with k: 5
Fitting model with k: 7
Fitting model with k: 9
Best K: 5, training final model


In [92]:
model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [80]:
print(f"Best AUC: {best_auc}, best K: {best_k}")

Best AUC: 0.6947423037011414, best K: 10


`cross_val_score` is probably the easiest way to call the outer loop.  
<https://scikit-learn.org/stable/modules/cross_validation.html#>
```python
from sklearn.model_selection import cross_val_score
ilf = our_inner_loop_function
scores = cross_val_score(ilf, X, y, cv=5)
scores
# array([0.96..., 1.  ..., 0.96..., 0.96..., 1.        ])
```

In [42]:
t, v = next(iter(inner.split(test_data['/expression'])))

In [43]:
x_train, y_train, x_valid, y_valid = test_data['/expression'].iloc[t], test_data['/labels'].iloc[t],\
test_data['/expression'].iloc[v], test_data['/labels'].iloc[v]

In [44]:
fitted_model1 = knn.fit(x_train, y_train)

In [59]:
train_prob = fitted_model1.predict_proba(x_train)
valid_prob = fitted_model1.predict_proba(x_valid)

In [67]:
train_acc = accuracy_score(y_true=y_train, y_pred=[np.argmax(e) for e in train_prob])
valid_acc = accuracy_score(y_true=y_valid, y_pred=[np.argmax(e) for e in valid_prob])
train_auc = roc_auc_score(y_true=y_train, y_score=train_prob[:,1])
valid_auc = roc_auc_score(y_true=y_valid, y_score=valid_prob[:,1])

In [70]:
print(f"Train acc: {train_acc}, valid acc: {valid_acc}\nTrain_auc: {train_auc}, valid_auc: {valid_auc}")

Train acc: 0.6651270207852193, valid acc: 0.5504587155963303
Train_auc: 0.7845793374019181, valid_auc: 0.5645706558485464
